Copyright (c) 2021, David Williams, Reza Sameni

All rights reserved.

This source code is licensed under the BSD-style license found in the
LICENSE file in the root directory of this source tree. 

## Import dependencies

In [ ]:
import numpy as np
import scipy.io as spio
import matplotlib.pyplot as plt

import os

os.chdir("../../Databases/SampleData")

sample = spio.loadmat("SampleECG128Hz.mat")
y = sample["y"]

os.chdir("../../Python3/Tools")
from AdaptiveFilter import AdaptiveFilter

## Set variables and call function

In [ ]:
fs = 128
ECG = np.array(y[:, 1])
N = len(ECG)
n = np.array(range(N))

In [ ]:
rr = 0.1
x = ECG + (1.21 * np.sin(100 * np.pi * n / fs)) + (rr * np.random.randn(1, N))
x = x[0]

In [ ]:
delay = 3
taps = 5
mu = 0.005
[ECG_estimate, Noise_estimate] = AdaptiveFilter(x, delay, taps, mu)

## Calculate SNR

In [ ]:
I = np.array(
    range(int(np.floor(2 * len(ECG) / 4)), int(np.ceil((3 * len(ECG) / 4) + 1)))
)

SignalPower = np.mean(ECG[I] ** 2)
PreNoisePower = np.mean((x[I] - ECG[I]) ** 2)
PostNoisePower = np.mean((ECG[I] - ECG_estimate[I]) ** 2)

Initial_SNR = 10 * np.log10(SignalPower / PreNoisePower)
AF_SNR = 10 * np.log10(SignalPower / PostNoisePower)

## Plot functions

In [ ]:
t = n / fs

plt.subplot(2, 1, 1)
plt.plot(t, x, label="Noisy")
plt.plot(t, ECG_estimate, label="ANC Output")
plt.plot(t, ECG, label="Original")
plt.xlabel("time (s)")
plt.ylabel("Amplitude")
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(t, Noise_estimate, label="Noise Estimate")
plt.xlabel("time (s)")
plt.ylabel("Amplitude")
plt.legend()